<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.4: A FIRRTL Transform Example

**Prev: [Common Pass Idioms](4.3_firrtl_common_idioms.ipynb)**<br>

이 AnalyzeCircuit Transform은 `firrtl.ir.Circuit`를 탐색하고 모듈당 찾은 추가 작업 수를 기록합니다.

## Setup

다음을 실행하십시오.

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
// Compiler Infrastructure

// Firrtl IR classes

// Map functions

// Scala's mutable collections
import scala.collection.mutable



## Counting Adders Per Module

앞에서 설명한 것처럼 Firrtl 회로는 트리 표현을 사용하여 표현됩니다.
   - Firrtl `Circuit`는 `DefModule`의 시퀀스를 포함합니다.
   - `DefModule`은 `Port`의 시퀀스를 포함하며 아마도 `Statement`일 수 있습니다.
   - '문'에는 다른 '문' 또는 '표현식'이 포함될 수 있습니다.
   - 'Expression'은 다른 'Expression'을 포함할 수 있습니다.

회로의 모든 Firrtl IR 노드를 방문하기 위해 이 트리를 재귀적으로 따라가는 함수를 작성합니다. 통계를 기록하기 위해 'Ledger' 클래스를 전달하고 추가 작업을 발견할 때 사용합니다.

In [ ]:
class Ledger {
  import firrtl.Utils
  private var moduleName: Option[String] = None
  private val modules = mutable.Set[String]()
  private val moduleAddMap = mutable.Map[String, Int]()
  def foundAdd(): Unit = moduleName match {
    case None => sys.error("Module name not defined in Ledger!")
    case Some(name) => moduleAddMap(name) = moduleAddMap.getOrElse(name, 0) + 1
  }
  def getModuleName: String = moduleName match {
    case None => Utils.error("Module name not defined in Ledger!")
    case Some(name) => name
  }
  def setModuleName(myName: String): Unit = {
    modules += myName
    moduleName = Some(myName)
  }
  def serialize: String = {
    modules map { myName =>
      s"$myName => ${moduleAddMap.getOrElse(myName, 0)} add ops!"
    } mkString "\n"
  }
}

이제 회로를 순회하고 가산기를 만날 때마다 '원장'을 업데이트하는 FIRRTL 변환을 정의해 보겠습니다(op 인수 'Add'가 있는 'DoPrim'). 지금은 `inputForm` 또는 `outputForm`에 대해 걱정하지 마십시오.

시간을 내서 `walkModule`, `walkStatement` 및 `walkExpression`이 FIRRTL AST의 모든 `DefModule`, `Statement` 및 `Expression` 노드를 순회할 수 있도록 하는 방법을 이해하십시오.

답변할 질문:
   - **walkModule이 walkExpression을 호출하지 않는 이유는 무엇입니까?**
   - **walkExpression이 후위 순회를 수행하는 이유는 무엇입니까?**
   - **Expression의 사전 주문 순회를 수행하도록 walkExpression을 수정할 수 있습니까?**

In [ ]:
class AnalyzeCircuit extends firrtl.Transform {
  import firrtl._
  import firrtl.ir._
  import firrtl.Mappers._
  import firrtl.Parser._
  import firrtl.annotations._
  import firrtl.PrimOps._
    
  // Requires the [[Circuit]] form to be "low"
  def inputForm = LowForm
  // Indicates the output [[Circuit]] form to be "low"
  def outputForm = LowForm

  // Called by [[Compiler]] to run your pass. [[CircuitState]] contains
  // the circuit and its form, as well as other related data.
  def execute(state: CircuitState): CircuitState = {
    val ledger = new Ledger()
    val circuit = state.circuit

    // Execute the function walkModule(ledger) on every [[DefModule]] in
    // circuit, returning a new [[Circuit]] with new [[Seq]] of [[DefModule]].
    //   - "higher order functions" - using a function as an object
    //   - "function currying" - partial argument notation
    //   - "infix notation" - fancy function calling syntax
    //   - "map" - classic functional programming concept
    //   - discard the returned new [[Circuit]] because circuit is unmodified
    circuit map walkModule(ledger)

    // Print our ledger
    println(ledger.serialize)

    // Return an unchanged [[CircuitState]]
    state
  }

  // Deeply visits every [[Statement]] in m.
  def walkModule(ledger: Ledger)(m: DefModule): DefModule = {
    // Set ledger to current module name
    ledger.setModuleName(m.name)

    // Execute the function walkStatement(ledger) on every [[Statement]] in m.
    //   - return the new [[DefModule]] (in this case, its identical to m)
    //   - if m does not contain [[Statement]], map returns m.
    m map walkStatement(ledger)
  }

  // Deeply visits every [[Statement]] and [[Expression]] in s.
  def walkStatement(ledger: Ledger)(s: Statement): Statement = {

    // Execute the function walkExpression(ledger) on every [[Expression]] in s.
    //   - discard the new [[Statement]] (in this case, its identical to s)
    //   - if s does not contain [[Expression]], map returns s.
    s map walkExpression(ledger)

    // Execute the function walkStatement(ledger) on every [[Statement]] in s.
    //   - return the new [[Statement]] (in this case, its identical to s)
    //   - if s does not contain [[Statement]], map returns s.
    s map walkStatement(ledger)
  }

  // Deeply visits every [[Expression]] in e.
  //   - "post-order traversal" - handle e's children [[Expression]] before e
  def walkExpression(ledger: Ledger)(e: Expression): Expression = {

    // Execute the function walkExpression(ledger) on every [[Expression]] in e.
    //   - return the new [[Expression]] (in this case, its identical to e)
    //   - if s does not contain [[Expression]], map returns e.
    val visited = e map walkExpression(ledger)

    visited match {
      // If e is an adder, increment our ledger and return e.
      case DoPrim(Add, _, _, _) =>
        ledger.foundAdd
        e
      // If e is not an adder, return e.
      case notadd => notadd
    }
  }
}

## Running our Transform

이제 정의했으므로 끌 디자인에서 실행해 보겠습니다. 먼저 Chisel 모듈을 정의하겠습니다.

In [ ]:
// Chisel stuff
import chisel3._
import chisel3.Input // Technicality: avoid a conflict with _root_.almond.input.Input
import chisel3.util._

In [ ]:
class AddMe(val nInputs: Int, val width: Int) extends Module {
  val io = IO(new Bundle {
    val in  = Input(Vec(nInputs, UInt(width.W)))
    val out = Output(UInt(width.W))
  })
  io.out := io.in.reduce(_ +& _)
}

다음으로 FIRRTL AST 구문으로 자세히 설명하겠습니다.

In [ ]:
val firrtlSerialization = chisel3.Driver.emit(() => new AddMe(8, 4))

마지막으로 FIRRTL을 Verilog로 컴파일하되 사용자 정의 변환을 컴파일에 포함시키자. 발견한 추가 작업의 수를 출력한다는 점에 유의하십시오!

**참고**(2021년 1월): [버그](https://github.com/freechipsproject/chisel-bootcamp/issues/129)로 인해 다음 줄이 깨질 수 있습니다.

In [ ]:
val verilog = compileFIRRTL(firrtlSerialization, new firrtl.VerilogCompiler(), Seq(new AnalyzeCircuit()))

`compileFIRRTL` 함수는 이 튜토리얼에서만 정의됩니다. 다음 섹션에서 customTransforms를 삽입하는 과정을 설명할 것입니다.

그것이 이 섹션에 대한 것입니다!